# Epsilon Selection

In this script, we will be testing several different epsilons as a kmapper parameter to determine how many states persist when as this value changes. We will first run the kmapper algorithm on the whole interpolated and cleaned LTRM water quality data set. We will then use code written by Killian to determine the density of each node, find local maximum, and determine local maxima states. We will count how many states are in each scomplex with the different choices of epsilon. Since the states, as of now, do not hold qualitative meaning, we will only be comparing state persistence as a count of how many there are and generally where they exist in the simplicial complex. Also note that the state analysis is run only on the largest shape in the simplicial complex.

## Import libraries

In [3]:
import kmapper as km
from kmapper.plotlyviz import *
import pandas as pd

import sklearn
#from sklearn import ensemble
# DBSCAN from sklearn for clustering algorithms
from sklearn.cluster import DBSCAN
# PCA from sklearn for projection/lens creation
from sklearn.decomposition import PCA
# from sklearn.manifold import MDS
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import RobustScaler

# scipy for interpolation
# import scipy 
# from scipy.interpolate import *
import hdbscan
import pickle as pk
import json as js
import matplotlib.pyplot as plt
from sklearn.compose import ColumnTransformer

# Plotly and Dash
import plotly.graph_objs as go
from kmapper.plotlyviz import plotlyviz
from kmapper.plotlyviz import *
from ipywidgets import (HBox, VBox)
import dash_html_components as html
import dash_core_components as dcc
import dash
from ipywidgets import interactive, HBox, VBox, widgets, interact
import warnings
warnings.filterwarnings("ignore")

## Read data

In [11]:
water20 = pd.read_csv("../../LTRM data/RF interpolation/water_full.csv")
#water20 = water20[["SHEETBAR", "DATE", "LATITUDE","LONGITUDE","FLDNUM", "STRATUM","LOCATCD",
#"TN","TP","TEMP", "DO", "TURB", "VEL", "SS", "WDP", "CHLcal",  "SECCHI", "YEAR", "SEASON"]]

## Robust Scaling

In [12]:
continuous_variables = ["TN","TP","TEMP", "DO", "TURB", "VEL", "SS", "WDP", "CHLcal", "SECCHI"]

ct = ColumnTransformer([('somename', RobustScaler(), continuous_variables)], remainder = 'passthrough')

water_df = pd.DataFrame(ct.fit_transform(water_20))

water_df = pd.DataFrame(water_df, columns = ["TN","TP","TEMP", "DO", "TURB", "VEL", "SS", "WDP", "CHLcal",  "SECCHI",
                                             "SHEETBAR", "DATE", "LATTITUDE","LONGITUDE","FLDNUM", "STRATUM","LOCATCD",
                                             "YEAR", "SEASON"])

NameError: name 'water_20' is not defined